## Design Pattern


### SOLID 원칙

객체지향 설계 5대 원칙 (SRP, OCP, LSP, ISP, DIP)을 의미한다.


#### 1. Single Responsibility Principle

단일 책임 원칙, 한 클래스는 단 하나의 책임만 가진다. 만약 여러 책임이 합쳐지면, 변경이나 유지보수가 어려워진다.

In [ ]:
class DataManager:
    def read_data(self, filename):
        # 파일에서 데이터 읽기
        pass

    def process_data(self, data):
        # 데이터 처리
        pass

    def save_report(self, data, out_file):
        # 처리 결과를 보고서 형태로 저장
        pass

위 클래스의 경우, 하나의 클래스 내에서 읽기와 프로세싱, 파일 저장까지 모두 이뤄지고 있다. 하나의 클래스에 여러개의 책임이 따르면, 직관성이 떨어지고 유지보수가 복잡해진다.

In [ ]:
class DataReader:
    def read_data(self, filename):
        # 파일에서 데이터만 읽는다
        pass

class DataProcessor:
    def process_data(self, data):
        # 데이터 처리만 한다
        pass

class ReportSaver:
    def save_report(self, data, out_file):
        # 결과를 저장만 한다
        pass

위 예시처럼 단일 책임으로 분리한다. 각각의 변경(입출력, 처리, 저장) 요구사항이 클래스별로 독립된다.

#### 2. OCP (Open/Closed Principle)

개방-폐쇄 원칙, 확장에는 열려 있고, 변경에는 닫혀 있어야 한다. 기존 코드를 수정하지 않고, 새로운 기능이나 구현을 추가해도 확장이 가능하도록 설계한다.

In [ ]:
class CommissionCalculator:
    def calculate_commission(self, trade_type, amount):
        if trade_type == "stock":
            return amount * 0.001
        elif trade_type == "option":
            return amount * 0.002

위 예시는 `trade_type`가 늘어날 때마다 `if/elif`를 수정해야 하므로 변경에 닫혀 있지 않다.

In [ ]:
from abc import ABC, abstractmethod

class CommissionStrategy(ABC): # 추상 클래스 
    @abstractmethod
    def calculate(self, amount):
        pass

class StockCommission(CommissionStrategy):
    def calculate(self, amount):
        return amount * 0.001

class OptionCommission(CommissionStrategy):
    def calculate(self, amount):
        return amount * 0.002

class CommissionCalculator:
    def __init__(self, strategy: CommissionStrategy):
        self.strategy = strategy

    def calculate_commission(self, amount):
        return self.strategy.calculate(amount)

In [ ]:
calculator = CommissionCalculator(StockCommission())
fee = calculator.calculate_commission(100_000)

새로운 대상을 추가할 때에는 클래스만 추가하면 되며, 기존 클래스 코드는 수정하지 않고도 확장된다.

#### 3. LSP (Liskov Substitution Principle)

리스코프 치환 원칙, 자식 클래스는 부모 클래스로 대체(치환)될 수 있어야 한다. 즉, 부모 클래스(인터페이스)가 기대하는 동작이나 규약을 자식 클래스도 위배하지 않아야 한다.

In [ ]:
class Asset:
    def get_price(self):
        pass

class Stock(Asset):
    def get_price(self):
        return 100  # 정상적인 숫자 반환

class Bond(Asset):
    def get_price(self):
        return None  # 규약에 어긋나는 반환 (가격 불가)

def print_asset_price(asset: Asset):
    price = asset.get_price()
    print(f"Price is {price}")

bond = Bond()
print_asset_price(bond)  # None 이 반환되어 로직상 에러가 날 수 있다

위 예시는 `Bond`가 부모(Asset)의 기대 동작(가격을 숫자로 반환)을 충족하지 않아, 부모 타입으로서 치환이 깨진다.

In [ ]:
class Asset:
    def get_price(self):
        pass

class Stock(Asset):
    def get_price(self):
        return 100

class Bond(Asset):
    def get_price(self):
        return 90  # 부모와 동일하게 숫자 반환

def print_asset_price(asset: Asset):
    price = asset.get_price()
    print(f"Price is {price}")

bond = Bond()
print_asset_price(bond)  # 정상적으로 90 출력

#### 4. ISP (Interface Segregation Principle)

인터페이스 분리 원칙, 덩치 큰 인터페이스 하나를 만들지 말고, 작고 구체적인 인터페이스 여러 개로 분리한다. 불필요한 메서드를 구현할 의무가 생기지 않도록 설계한다.

In [ ]:
class TradingOperations:
    def buy_stock(self):
        pass
    def sell_stock(self):
        pass
    def write_option(self):
        pass
    def exercise_option(self):
        pass
    # 모든 트레이딩 관련 메서드를 하나에 다 넣었다

class StockTrader(TradingOperations):
    def buy_stock(self):
        pass
    def sell_stock(self):
        pass
    def write_option(self):
        pass  # 사용하지 않는 기능도 구현해야 함
    def exercise_option(self):
        pass  # 사용하지 않는 기능도 구현해야 함

위 경우, `StockTrader`는 옵션 관련 메서드를 전혀 쓸 일이 없지만, 인터페이스에 포함된 탓에 구현해야 한다.

In [ ]:
from abc import ABC, abstractmethod

class StockTrading(ABC):
    @abstractmethod
    def buy_stock(self):
        pass
    @abstractmethod
    def sell_stock(self):
        pass

class OptionTrading(ABC):
    @abstractmethod
    def write_option(self):
        pass
    @abstractmethod
    def exercise_option(self):
        pass

class StockTrader(StockTrading):
    def buy_stock(self):
        pass
    def sell_stock(self):
        pass

class OptionTrader(OptionTrading):
    def write_option(self):
        pass
    def exercise_option(self):
        pass

위 코드처럼 필요한 인터페이스만 구현하도록 분리하면 필수 기능만을 공유하도록 설계할 수 있다.

#### 5. DIP (Dependency Inversion Principle)

의존성 역전 원칙, 상위(고수준) 모듈이 하위(저수준) 모듈 세부 구현에 의존하지 않도록 하고, 추상화(인터페이스)에 의존하게 만든다.

In [ ]:
class FileLogger:
    def write_log(self, msg):
        with open("log.txt", "a") as f:
            f.write(msg + "\n")

class TradingSystem:
    def __init__(self):
        self.logger = FileLogger()  # 구체 클래스 직접 의존

    def trade(self, info):
        # 트레이딩 로직
        self.logger.write_log("Trade executed: " + info)

위 경우 `TradingSystem`은 `FileLogger`라는 구현에 직접 의존한다. 만약 데이터베이스에 로그를 쓰도록 바꾸려면 `TradingSystem` 코드를 수정해야 한다.

In [ ]:
from abc import ABC, abstractmethod

class Logger(ABC):
    @abstractmethod
    def write_log(self, msg):
        pass

class FileLogger(Logger):
    def write_log(self, msg):
        with open("log.txt", "a") as f:
            f.write(msg + "\n")

class DbLogger(Logger):
    def write_log(self, msg):
        # DB에 로그 저장
        pass

class TradingSystem:
    def __init__(self, logger: Logger):
        self.logger = logger  # 추상화(인터페이스)에 의존

    def trade(self, info):
        # 트레이딩 로직
        self.logger.write_log("Trade executed: " + info)

In [ ]:
ts = TradingSystem(FileLogger())
ts.trade("AAPL BUY 100")

위 예시에서는 `TradingSystem`이 Logger 인터페이스에 의존한다. 구체 로거를 교체해도 상위 모듈 코드를 수정할 필요가 없다.

### Design Pattern

OOP에서 자주 사용되는 해결책 패턴을 의미한다. 자주 발생하는 설계 문제를 효율적으로 해결하고, 가독성과 유지보수 및 원활한 협업을 목표로 한다.

#### 1. Singleton

- 애플리케이션에서 특정 클래스의 인스턴스가 단 하나만 존재하도록 보장한다.
- 인스턴스가 여러 개 생기면 안 되거나, 전역적으로 공유되는 리소스(예: 설정값, 로거) 등에 주로 사용한다.

#### 2. Factory Method

- 객체 생성 로직을 서브클래스에 위임하는 방식이다.
- 상위 클래스(또는 인터페이스)는 객체를 생성하는 **‘팩토리 메서드’** 를 갖고, 실제 구현은 자식 클래스가 결정한다.
- 새로운 구체 클래스가 추가되어도 상위 클래스 코드를 수정하지 않고 확장할 수 있다.

#### 3. Strategy

- 알고리즘(또는 행위)을 캡슐화하여 런타임에 교체 가능하도록 만든다. 예를 들어, 로직을 여러 가지 전략으로 구분하고, 상황에 따라 다른 전략 객체를 주입해 사용한다.

#### 4. Observer

- **주체(Subject)** 와 이를 관찰하는 옵저버(Observer) 간 일 대 다 의존 관계를 설정한다.
- 주체의 상태가 변하면, 연결된 옵저버들에게 자동으로 알림을 보낸다.
- 이벤트 기반 시스템, GUI, 금융 시세 업데이트 등에서 흔히 사용한다.

#### 5. Decorator

- 기존 객체에 새로운 기능을 동적으로 추가하는 방법이다.
- 상속 대신 **랩핑(Wrapping)** 을 통해 객체에 책임을 덧붙일 수 있다.